# 영화 평점 긍정 부정 예측 AI 개발하기

# 데이터 파일 읽기
--------------------------------------------
0. 데이터 다운로드(in kaggle): https://www.kaggle.com/c/word2vec-nlp-tutorial/data
1. 긍정과 부정 데이터가 있는 train 데이터를 읽어온다.
2. 긍정과 부정이 없는 test데이터를 읽어온다.
2. 판다스 라이브러리 사용함

In [1]:
import pandas as pd

"""
header = 0 은 파일의 첫 번째 줄에 열 이름이 있음을 나타내며 
delimiter = \t 는 필드가 탭으로 구분되는 것을 의미한다.
quoting = 3은 쌍따옴표를 무시하도록 한다.
"""
# QUOTE_MINIMAL (0), QUOTE_ALL (1), 
# QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).

# 레이블인 sentiment 가 있는 학습 데이터
train = pd.read_csv('labeledTrainData.tsv', delimiter='\t', quoting=3)
# 레이블이 없는 테스트 데이터
test = pd.read_csv('testData.tsv', delimiter='\t', quoting=3)
test.shape

(25000, 2)

# 자료를 눈으로 확인하기
----------------------------------------
1. 센티멘트가 있는 train데이터를 살펴보기
2. head()사용: 위에서 원하는 만큼 데이터를 찍어보는 함수
3. train데이터는 sentiment가 있음을 확인.

In [2]:
train.head(5)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
# 레이블인 'sentiment'가 없다. 이 데이터를 기계학습을 통해 예측한다.
test.head(5)

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


# 데이터 정제하기: 단어장 만들기
------------------------------------------
1. 문서는 다양한 단어로 이루어져있음
2. 우리는 일종의 단어장을 만들어 볼 것이다. 
3. 우리가 중학교 때 영어 사전을 들고 다니지 않고 들고다녔던 단어장을 기억해보자!
4. 단어장에는 중복된 단어가 없어야한다.   

### 아래의 세 문장을 보자
--------------------------------------------------
```
문장1: The car is expensive.    
문장2: The truck is cheap.    
문장3: The car is expensive and the truck is cheap.   
```
------------------------------------------------------
__문장에 등장하는 단어는 7개이며 아래와 같은 알파벳 순서의 단어장을 만들어 볼 수 있다__
```
[and, car, cheap, expensive, is, the, truck]
```


# 단어장 만들기: 텍스트를 벡터로 만들기
------------------------------------------
1. 단어장의 단어를 인덱스로 바꾸기
>- and -> 0, car -> 1, ... truck -> 6으로 만들기
>- 문장1~문장3에 등장하는 단어의 빈도수를 적어 정리하기
>- 문장1~문장3의 특성을 나타내는 표로 배열 즉, 벡터로 구성함
2. scikit-learn의 CounterVectorizer: 위에서 설명한 텍스트 문장을 벡터로 구성하는 것을 해주는 클래스

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

txt1= 'the car is expensive'
txt2= 'the truck is cheap'
txt3= 'the car is expensive and the truck is cheap'

count= CountVectorizer()
docs = np.array([txt1, txt2, txt3])
bag = count.fit_transform(docs)

print(bag.toarray())

[[0 1 0 1 1 1 0]
 [0 0 1 0 1 1 1]
 [1 1 1 1 2 2 1]]


# 단어장 만들기: tf-idf기법 사용하여 빈도수 체크하기
-----------------------------------------
1. tf-idf기법이란?
>- 문장에서 등장하는 단어가 훈련에 쓸만한 정보를 가지고 있는지 빈도수를 통해 판단하기 위해서 사용함.
>- 영어 단어 'is'와 같은 be동사에 적용됨: 등장하는 빈도수는 높지만 큰 의미를 둘만한 단어가 아님
>- 공식: tf-idf(t,d) tf(t,d) * (idf(t,d)+1)
>- sklearn은 tf.idf(t,d)계산을 위해 TfidTrnasformer()클래스를 제공함
2. sklern의 TfidTransformer클래스를 라이브러리에 추가함.

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
np.set_printoptions(precision=2)# tf-idf(t,d)값의 L2정규화 값을 계산
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.56 0.   0.56 0.43 0.43 0.  ]
 [0.   0.   0.56 0.   0.43 0.43 0.56]
 [0.4  0.31 0.31 0.31 0.48 0.48 0.31]]


__결과를 보면 0과 1의 값이 tf-idf를 적용한 뒤 소수점으로 다른 값이 나오는 것을 확인할 수 있다. 이렇게 바뀐 이유는 tf-idf가 문장에서의 특정 단어의 빈도수로 바꾸어주었기 때문이다__

# 문장을 컴퓨터가 읽기 편하게 만들기
------------------------------------------------------
#### step1: html 태그 제거
#### step2: 영문자가 아닌 문자는 공백으로 변환
#### step3: 소문자 변환
#### step4: 동사를 동사원형으로 만들기
#### step5: 단어들을 합쳐서 문장으로 다시 재결합

In [6]:
# html 태그가 섞여있기 때문에 이를 정제해줄 필요가 있음
train['review'][0][:700]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely lik'

In [7]:
import re
import pandas as pd
from time import time

def preprocessor(text):
    #특수기호, HTML 테그 등 제거
    text = re.sub('<[^>]*>','',text)
    text= re.sub('[^a-zA-Z]',' ',text)
    text = text.lower() 
    return text

train = pd.read_csv('labeledTrainData.tsv', delimiter='\t', quoting=3)

stime = time()
print('전처리 시작')
train['review'] = train['review'].apply(preprocessor)
print('전처리 완료: 소요시간 [%d]초' %(time()-stime))

train.to_csv('refined_labeledTrainData.csv', index=False)

전처리 시작
전처리 완료: 소요시간 [3]초


In [8]:
import pandas as pd

df = pd.read_csv('refined_labeledTrainData.csv')
df.head()

,id,sentiment,review
0,"""5814_8""",1,with all this stuff going down at the moment ...
1,"""2381_9""",1,the classic war of the worlds by timothy ...
2,"""7759_3""",0,the film starts with a manager nicholas bell...
3,"""3630_4""",0,it must be assumed that those who praised thi...
4,"""9495_8""",1,superbly trashy and wondrously unpretentious ...


__review에 문장부호가 사라졌고 모두 소문자로 바뀐 것을 볼 수 있다__

# 문장을 단어로 쪼개기
-------------------------------------------------
1. nltk를 설치함
>- 문장에서 단어를 분리하는 파이썬 확장 라이브러리
>- 자연어 처리와 관련된 다양한 api제공
>- 다음 사이트를 설명을 참고하여 설치할 것http://ling.snu.ac.kr/class/cl_under1801/InstallationGuideforWindowUsers.pdf
2. tokenizer(), tokernizer_porter()함수로 각각 공백으로 단어를 분리하여 리스트로 리턴한다

In [9]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

porter = PorterStemmer()
stop = stopwords.words('english')

#공백으로 단어 분리
def tokenizer(text):
    return text.split()

#Porter Stemming 알고리즘을 이용해 단어분리
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# 전체 데이터로 학습 모델 만들기
---------------------------------------------
1. 25000개의 리뷰 데이터에 모두 적용하기

2. scikit-learn에서 CountVectorizer()과 TfidTransformer()의 기능을 합쳐놓은 TfidVectorizer()클래스를 사용하기
>- 영화 리뷰 문장을 특정 벡터로 구성하고
>- 로지스틱 회귀 알고리즘으로 머신러닝 수행하기

3. 완성된 모델은 pklObject이름의 파일로 저장함

In [ ]:
#입력받은 리스트에 정의된 함수를 순차적으로 적용해주는 라이브러리
from sklearn.pipeline import Pipeline 
#로지스틱 회귀 알고리즘 라이브러리
from sklearn.linear_model import LogisticRegression 
#CountVectorizer()과 TfidTrnasformer()을 sklearn의 클래스 
from sklearn.feature_extraction.text import TfidfVectorizer
#특정 벡터로 구성하는 라이브러리
from sklearn.metrics import accuracy_score
#만든 모델을 파일로 저장해주는 라이브러리
import pickle
import os
from time import time
import pandas as pd
#정제했던 데이터 불러오기
df = pd.read_csv('refined_labeledTrainData.csv')
#전처리된 영화 리뷰 train데이터 15000개를 이용해 머신러닝을 수행함

# 머신러닝의 결과는 10000개의 test데이터로 검증해봄
X_train = df.loc[:15000,'review'].values
Y_train = df.loc[:15000,'sentiment'].values
X_test = df.loc[10000:,'review'].values
Y_test = df.loc[10000:, 'sentiment'].values

#tokenizer()로 단어를 공백으로 분리하고 TfidVectorizer()에 넣어줌
tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
#pipeline의 parameter은 (함수별명, 함수)같은 튜플로 만들어야 함
#따라서 함수 별명이 vect인 lr_tfidf인 튜플과
#clf가 별명인 tfidf튜플을 넣어준다. 
Ir_tfidf = Pipeline([('vect',tfidf),('clf',LogisticRegression(C = 10.0, penalty = 'l2', random_state = 0))])

#시간 계산 함수
stime = time()
print('머신러닝 시작')
#X_train과 Y_train을 이용하여 머신러닝을 수행함.
Ir_tfidf.fit(X_train, Y_train)
print('머신러닝 종료')

Y_pred = Ir_tfidf.predict(X_test)
print('테스트 종료: 소요시간 [%d]초' %(time()-stime))
print('정확도: %.3f' %accuracy_score(Y_test, Y_pred))

#머신러닝 결과를 pickle 모듈의 dump()를 이용하여 파일로 저장함
curDir = os.getcwd()
dest = os.path.join(curDir,'data','pklObject')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(Ir_tfidf,open(os.path.join(dest,'classifier.pkl'),'wb'),protocol=4)
print('머신러닝 데이터 저장 완료')

머신러닝 시작


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


머신러닝 종료
테스트 종료: 소요시간 [12]초
정확도: 0.925
머신러닝 데이터 저장 완료


# 사용자로부터 입력받은 값을 예측

In [ ]:
import pickle
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

df = pd.read_csv('refined_labeledTrainData.csv')

X_train = df.loc[:15000,'review'].values
Y_train = df.loc[:15000,'sentiment'].values
X_test = df.loc[10000:,'review'].values
Y_test = df.loc[10000:, 'sentiment'].values

curDir = os.getcwd()
clf = pickle.load(open(os.path.join(curDir,'data','pklObject','classifier.pkl'),'rb'))

Y_pred = clf.predict(X_test)
print('테스트 정확도: %.3f' %accuracy_score(Y_test, Y_pred))

label = {0: '부정적 의견', 1:'긍정적 의견'}

while True:
    txt = input("영문으로 리뷰를 작성하세요: ")
    if txt == '':
        break
        
    example = [txt]
    print('예측: %s\n확률: %.3f%%' %(label[clf.predict(example)[0]], np.max(clf.predict_proba(example))*100))
    

테스트 정확도: 0.925
영문으로 리뷰를 작성하세요: i've never seen a great movie.
예측: 긍정적 의견
확률: 99.960%
영문으로 리뷰를 작성하세요: Never Look Away fills its protracted running time with the absorbing story of an incredible life -- and its impact on the singular artist who lived it.
예측: 긍정적 의견
확률: 95.426%
영문으로 리뷰를 작성하세요: Everybody Knows is somewhat less than the sum of its parts despite the efforts of an outstanding cast - and a disappointing step back for writer-director Asghar Farhadi.
예측: 부정적 의견
확률: 83.059%
영문으로 리뷰를 작성하세요: i hated this movie.
예측: 부정적 의견
확률: 98.556%
